# WEBINAR - Przetwarzanie języka naturalnego, 16 lipca 2020

SAGES - NLP masterclass

Łukasz Kobyliński, Ryszard Tuora


2 modele do języka polskiego w spaCy:

IPI PAN - bardziej rozbudowany, wolniejszy, bardziej złożona instalacja - http://zil.ipipan.waw.pl/SpacyPL

model oficjalny - prostszy, znacznie szybszy, prosta instalacja - https://spacy.io/models/pl

model IPI PAN dla języka polskiego

składa się z:
- taggera morfosyntaktycznego
- lematyzatora
- parsera zależnościowego
- komponentu NER
- flexera (komponentu do fleksji)

In [ ]:
# Przygotowanie środowiska, komendy linux
# instalacja Morfeusza 2
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja spaCy

!python3 -m pip install spacy

# 1. instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz' pl_spacy_model_morfeusz-0.1.0.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.0.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

# 2. instalacja oficjalnego modelu spaCy
#!python3 -m spacy download pl_core_news_lg # wymaga spacy 2.3.0

# dodatkowe zależności:
!python3 -m pip install tqdm

In [1]:
### PYTHON 3
import keras
# ładowanie modelu
import spacy

nlp = spacy.load("pl_spacy_model_morfeusz")
#nlp = spacy.load("pl_core_news_lg")

# Część zero - Tokenizacja i reprezentacja tekstów

Wejściem do modelu są łańcuchy tekstowe (stringi), na wyjściu dostajemy obiekt reprezentujący struktury wykryte w tekście. Pierwszym krokiem który musi być wykonany aby przetworztć tekst, jest tokenizacja, czyli podział tekstu na tokeny/segmenty. Tokeny w większości przypadków odpowiadają słowom "od spacji do spacji". Ale warto zwrócić uwagę na kilka przypadków odstających od takiej prostej reguły.

Wynikiem potoku spaCy jest obiekt Doc, który składa się z obiektów Token.

In [2]:
txt = "Chciałby, żebym pojechał do miasta z zielono-żółto-białą flagą (np. Zielonej Góry)."
split = txt.split()
doc = nlp(txt)
print("spaCy          .split()\n")
for i in range(max([len(split), len(doc)])):
  try:
    tok1 = doc[i]
  except IndexError:
    tok1 = ""
  try:
    tok2 = split[i]
  except IndexError:
    tok2 = ""
  print("{0:15} {1:15}".format(tok1.orth_, tok2))

spaCy          .split()

Chciał          Chciałby,      
by              żebym          
,               pojechał       
żeby            do             
m               miasta         
pojechał        z              
do              zielono-żółto-białą
miasta          flagą          
z               (np.           
zielono         Zielonej       
-               Góry).         
żółto                          
-                              
białą                          
flagą                          
(                              
np                             
.                              
Zielonej                       
Góry                           
)                              
.                              


spaCy pozwala na zapisywanie przetworzonych dokumentów w formacie JSON

In [3]:
import json

doc_json = doc.to_json()
json_string = json.dumps(doc_json, indent=2, ensure_ascii=False)
print(json_string)

{
  "text": "Chciałby, żebym pojechał do miasta z zielono-żółto-białą flagą (np. Zielonej Góry).",
  "ents": [
    {
      "start": 68,
      "end": 81,
      "label": "placeName"
    }
  ],
  "sents": [
    {
      "start": 0,
      "end": 83
    }
  ],
  "tokens": [
    {
      "id": 0,
      "start": 0,
      "end": 6,
      "pos": "VERB",
      "tag": "praet",
      "dep": "ROOT",
      "head": 0
    },
    {
      "id": 1,
      "start": 6,
      "end": 8,
      "pos": "PART",
      "tag": "qub",
      "dep": "aux:cnd",
      "head": 0
    },
    {
      "id": 2,
      "start": 8,
      "end": 9,
      "pos": "PUNCT",
      "tag": "interp",
      "dep": "punct",
      "head": 5
    },
    {
      "id": 3,
      "start": 10,
      "end": 14,
      "pos": "SCONJ",
      "tag": "comp",
      "dep": "mark",
      "head": 5
    },
    {
      "id": 4,
      "start": 14,
      "end": 15,
      "pos": "AUX",
      "tag": "aglt",
      "dep": "aux:clitic",
      "head": 5
    },
    {
   

# Część pierwsza - podobieństwo w języku
Sercem modelu jest reprezentacja wektorowa słów, 500 tys. wektorów wyekstrahowanych z embeddingów word2vec KGR10.

###Bardzo popularny przykład "arytmetyki słów":
znaczenie słów jest reprezentowane przez wektory, dla których mamy zdefiniowane operacje matematyczne. Możemy więc odjąć od znaczenia słowa "królowa", znaczenie słowa "kobieta", i dodać doń znaczenie słowa "mężczyzna", licząc iż wektor będący wynikiem takiego działania odpowiada słowu "król". W praktyce wektor taki najprawdopodobniej nie ma interpretacji, ale możemy znaleźć najbliższy wektor który ma jakąkolwiek interpretację przeszukując słownik.

In [23]:
# od wersji 2.3 wektory są ładowane "leniwie"
# musimy więc najpierw wymusić załadowanie wszystkich wektorów

for s in nlp.vocab.vectors:
    _ = nlp.vocab[s]

In [24]:
from scipy import spatial
 
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
 
man = nlp.vocab['mężczyzna'].vector
woman = nlp.vocab['kobieta'].vector
queen = nlp.vocab['królowa'].vector
king = nlp.vocab['król'].vector
 

maybe_king = man - woman + queen
computed_similarities = []
 
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue
 
    similarity = cosine_similarity(maybe_king, word.vector)
    computed_similarities.append((word, similarity))
 
computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in computed_similarities[:10]])

['książę', 'król', 'królewicz', 'monarcha', 'hrabia', 'władca', 'Książę', 'arcyksiążę', 'książe', 'Arcyksiążę']


In [25]:
w1 = "pies"
w2 = "psami"
w3 = "zwierzę"
w4 = "buldog"
w5 = "obroża"
w6 = "smycz"
w7 = "marchewka"
w8 = "słońce"

def similarity(w1, w2):
  w1_lex = nlp.vocab[w1]
  w2_lex = nlp.vocab[w2]
  if w1_lex.has_vector and w2_lex.has_vector:
    sim = w1_lex.similarity(w2_lex)
    print("{} vs. {} -> {}".format(w1, w2, sim))
  else:
    print("Jedno ze słów nie ma reprezentacji wektorowej.")
for w in [w2, w3, w4, w5, w6, w7, w8]:
  similarity(w1, w)

pies vs. psami -> 0.6705560088157654
pies vs. zwierzę -> 0.684103786945343
pies vs. buldog -> 0.8052963018417358
pies vs. obroża -> 0.5760606527328491
pies vs. smycz -> 0.5837645530700684
pies vs. marchewka -> 0.4573368728160858
pies vs. słońce -> 0.2709254324436188


## Zadanie 1:

nlp.vocab może być traktowany jako iterator (stanowi słownik, choć nie wszystkim słowom przypisane są wektory). Napisz funkcję thesaurus(word) która dla podanego słowa znajduje dziesięć najbardziej podobnych słów. Pamiętaj o złożoności obliczeniowej sortowania! 

Możesz wykorzystać lematyzację, oraz tagowanie morfosyntaktyczne, by pozbyć się zbędnych słów.

###spaCy umożliwia liczenie podobieństwa między słowami 
####jest ono proporcjonalne do cosinusa kąta wektorami je reprezentującymi. Odpowiednia funkcja jest zdefiniowana dla leksemów (tutaj oznaczają one słowa z pominięciem kontekstu).

In [26]:
def thesaurus(word):
  synonyms = []
  minimum = 0
  w_lex = nlp.vocab[word]
  for candidate in nlp.vocab:
    if candidate.has_vector and candidate != w_lex:
      similarity = w_lex.similarity(candidate)
      if similarity > minimum:
        synonyms.append((similarity, candidate.orth_))
      if len(synonyms) >10:
        synonyms = sorted(synonyms, reverse = True)
        synonyms = synonyms[:-1]
        minimum = synonyms[-1][0]

  return synonyms

syn = (thesaurus("pies"))
for x in syn:
  print(x)

(0.9248929, 'piesek')
(0.9197937, 'szczeniak')
(0.9130602, 'Pies')
(0.9057689, 'kot')
(0.9051197, 'wilczur')
(0.9018397, 'zwierzak')
(0.9007868, 'psiak')
(0.8871712, 'owczarek')
(0.88386726, 'czworonóg')
(0.87603015, 'jamnik')


## Streszczanie

## Dezambiguacja semantyczna



# Część druga - Tagowanie morfosyntaktyczne
korzystamy z tagsetu NKJP
Nasz tagger to słownikowy tagger Morfeusz2 + dezambiguacja za pomocą neuronowego Toyggera (biLSTM)

Każdy token t ma trzy interesujące nas atrybuty: 
- t.tag_ : klasa gramatyczna według polskiego tagsetu NKJP (http://nkjp.pl/poliqarp/help/ense2.html)
- t.pos_ : klasa gramatyczna według międzynarodowego tagsetu UD (mapowana z NKJP)
- t._.feats : customowy atrybut odpowiadający cechom morfosyntaktycznym (np. rodzajowi gramatycznemu, lub liczbie), poszczególne wartości cech są oddzielone dwukropkiem

In [4]:
txt = "Nornica prowadzi zmierzchowo-nocny tryb życia, ale wychodzi również za dnia w poszukiwaniu pokarmu."
doc = nlp(txt) # przetworzenie textu przez pipeline, na wyjściu dostajemy iterowalny obiekty klasy Doc, przechowujący tokeny
for t in doc:
  print("{0:15} {1:8} {2:6} {3:15}".format(t.orth_, t.tag_, t.pos_, t._.feats)) # wypisujemy interpretację morfosyntaktyczną każdego tokenu

Nornica         subst    NOUN   sg:nom:f       
prowadzi        fin      VERB   sg:ter:imperf  
zmierzchowo     adja     ADJ                   
-               interp   PUNCT                 
nocny           adj      ADJ    sg:acc:m3:pos  
tryb            subst    NOUN   sg:acc:m3      
życia           subst    NOUN   sg:gen:n       
,               interp   PUNCT                 
ale             conj     CCONJ                 
wychodzi        fin      VERB   sg:ter:imperf  
również         qub      PART                  
za              prep     ADP    gen            
dnia            subst    NOUN   sg:gen:m3      
w               prep     ADP    loc:nwok       
poszukiwaniu    ger      NOUN   sg:loc:n:imperf:aff
pokarmu         subst    NOUN   sg:gen:m3      
.               interp   PUNCT                 


##Zadanie 1:


![alt text](https://github.com/ryszardtuora/webinar_resources/raw/master/czesci_mowy.png)

Źródło: Irena Kamińska-Szmaj, *Różnice leksykalne między stylami funkcjonalnymi polszczyzny pisanej: Analiza statystyczna na materiale słownika frekwencyjnego*, 1990.

Przetwórz tekst znajdujący się w zmiennej txt, oblicz proporcję czasowników (używając tagów UD), na podstawie tego oszacuj gatunek do którego należy tekst.

In [21]:
import requests

txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/master/1.txt").text

doc = nlp(txt)
nouns = 0
verbs = 0
tokens = 0
non_interp = 0
for t in doc:
  tokens += 1
  if t.pos_ != "PUNCT":
    non_interp +=1
  if t.pos_ == "NOUN":
    nouns += 1
  if t.pos_ == "VERB":
    verbs += 1
    
print("Stosunek czasowników do rzeczowników: ", verbs/nouns)
print("Proporcja czasowników: ", verbs/non_interp)
print("Proporcja rzeczowników: ", nouns/non_interp)

Stosunek czasowników do rzeczowników:  0.0
Proporcja czasowników:  0.0
Proporcja rzeczowników:  0.3333333333333333


### Fleksja

Flexer pozwala na odmianę pojedynczych tokenów, do pożądanej charakterystyki morfologicznej. Argumentami do flexera jest słowo do odmiany (string, lub lepiej otagowany token), i przedzielony dwukropkami string znaczników morfosyntaktycznych.

Flexer umożliwia np. wypełnianie szablonów tekstów.

In [5]:
#filizanka = nlp("filiżanka")[0]
#print(filizanka._.feats)
filizanka = "filiżanka"
flexer = nlp.get_pipe("flexer")

tmpl1 = "Szukam szarej, ceramicznej {}.".format(flexer.flex(filizanka, "gen"))
tmpl2 = "Marzę o szarej, ceramicznej {}.".format(flexer.flex(filizanka, "loc"))
tmpl3 = "Szukam kompletu ceramicznych {}.".format(flexer.flex(filizanka, "gen:pl"))
print(tmpl1)
print(tmpl2)
print(tmpl3)

Szukam szarej, porcelanowej filiżanki.
Marzę o szarej, porcelanowej filiżance.
Szukam kompletu porcelanowych filiżanek.


In [ ]:
Flexer pozwala także na odmianę fraz wielowyrazowych (multi-word expressions - MWE).
W tym celu możemy do niego podać string, lub token odpowiadający głowie frazy.
Korzystamy z metody .flex_mwe()

In [12]:
filizanka2 = "biała, porcelanowa filiżanka z Chin"

tmpl1 = "Szukam {}.".format(flexer.flex_mwe(filizanka2, "gen"))
tmpl2 = "Marzę o {}.".format(flexer.flex_mwe(filizanka2, "loc"))
tmpl3 = "Szukam kompletu {}.".format(flexer.flex_mwe(filizanka2, "gen:pl"))

print(tmpl1)
print(tmpl2)
print(tmpl3)

Szukam białej, porcelanowej filiżanki z Chin.
Marzę o białej, porcelanowej filiżance z Chin.
Szukam kompletu białych, porcelanowych filiżanek z Chin.


## Zadanie 2

Zaprojektuj język znaczników morfosyntaktycznych dla luk w szablonach, który określa jaką formę muszą przyjąć frazy, by móc wypełnić te luki.
Zaprojektuj funkcję, która pobiera jako argumenty 1. szablon z tak oznakowaną luką, 2. frazę do wypełnienia luki, która odmienia wskazaną frazę, i wkleja ją w miejsce luki. 
Możesz założyć, że funkcja ma działać wtedy, i tylko wtedy, gdy w szablonie znajduje się jedna luka.

In [16]:
import re

gap_tag = "#gen:loc#"
gap_regex = re.compile("#[^#]+#")
template = "Przyglądam się #dat:pl#."
phrase = "czarne, skórzana torebka ze sprzączką"

def fill_template(template, phrase):
    match = gap_regex.search(template)
    tag = match.group(0)
    morpho = tag.strip("#")
    flexed = flexer.flex_mwe(phrase, morpho)
    filled = template.replace(tag, flexed)
    return filled

print(fill_template(template, phrase))

Przyglądam się czarnym, skórzanym torebkom ze sprzączką.


#Część druga - Lematyzacja i własności leksykalne
Nasz model umożliwia słownikową lematyzację przy pomocy Morfeusza, do dezambiguacji (tutaj np. rozróżnienia między "Głos zabrały mamy dzieci."-> "mama" i "My mamy samochód." -> "mieć" służy output taggera).

Lematyzacja pozwala redukować redundancję informacyjną, i ułatwiać zadania takie jak streszczanie, i przeszukiwanie.

Każdy z tokenów dodatkowo jest oznaczony ze względu na pewne własności leksykalne, np. :
- t.lemma_ - lemat - forma reprezentatywna danego leksemu
- t.is_stop - słowo należy do stoplisty (listy słów występujących najczęściej, a więc najmniej istotnych semantycznie)
- t.is_oov - słowo znajduje się poza słownikiem, i.e. embeddingami wykorzystanymi w modelu
- t.like_url - token ma strukturę url-a
- t.like_num - token jest liczbą
- t.is_alpha - token składa się tylko ze znaków alfabetycznych
- t.rank - miejse w rankingu częstości słów

In [17]:
txt = "Zdenerwowany gen. Leese mówił przez telefon swym podwładnym walczącym pod Monte Cassino, że rozmawia z nimi ze schronu."
doc = nlp(txt)
print("{0:16} {1:16} {2:5} {3:5} {4:20}\n".format("forma", "lemat", "OOV", "STOP", "Częstość"))
for t in doc:
  print("{0:16} {1:16} {2:5} {3:5} {4:20}".format(t.orth_, t.lemma_, t.is_oov, t.is_stop, t.rank)) # orth_ to atrybut odpowiadający formie słowa występującej w tekście

forma            lemat            OOV   STOP  Częstość            

Zdenerwowany     zdenerwować          0     0               117789
gen              generał              0     0                 4239
.                .                    0     0                    1
Leese            Leese                1     0 18446744073709551615
mówił            mówić                0     0                  536
przez            przez                0     1                   32
telefon          telefon              0     0                 1629
swym             swój                 0     0                 1994
podwładnym       podwładny            0     0                69725
walczącym        walczyć              0     0                58343
pod              pod                  0     1                   84
Monte            Monte                0     0                15050
Cassino          Cassino              0     0                33875
,                ,                    0     0                

##Zadanie 2:

Przetwórz tekst spod zmiennej txt, przekonwertuj go do listy lematów, usuń słowa ze stoplisty, oraz interpunkcję, wypisz dziesięć najczęściej pojawiających się lematów. Wypróbuj także opcję w której uwzględniamy tylko rzeczowniki


In [18]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/webinar_resources/master/2.txt").text

doc = nlp(txt)
words = {}
for t in doc:
  if not (t.is_stop or t.tag_ == "interp"):
    try:
      words[t.lemma_] +=1
    except KeyError:
      words[t.lemma_] = 1
listed = [(words[w],w) for w in words]
srt = sorted(listed, reverse = True)
print(srt[0:10])

NameError: name 'requests' is not defined

#Część trzecia - parsowanie zależnościowe

spaCy zawiera parser zależnościowy oparty o metodologię transition-based dependency parsing. 
Interesują nas tu cztery atrybuty:
 - t.head - link do tokenu będącego nadrzędnikiem tokenu t
 - t.dep_ - etykieta opisująca rodzaj zależności
 - t.subtree - generator opisujący poddrzewo rozpięte przez token t
 - t.ancestors - generator opisujący wszystkie przechodnie nadrzędniki tokenu t


Opis systemu etykiet: https://universaldependencies.org/u/dep/all.html

In [ ]:
txt = "Pierwsza wzmianka o Gdańsku pochodzi ze spisanego po łacinie w 999 Żywotu świętego Wojciecha."
doc = nlp(txt)

import pandas as pd

table = []
for tok in doc:
  tok_dic = {"form": tok.orth_, "label": tok.dep_, "head": tok.head.orth_, "subtree": list(tok.subtree), "ancestors": list(tok.ancestors)}
  table.append(tok_dic)
df = pd.DataFrame(table)
print(df.to_string())

### spaCy posiada wbudowaną wizualizację drzew zależnościowych

In [ ]:
from spacy import displacy

displacy.render(doc, jupyter = True)

###Poniższa funkcja służy łatwej wizualizacji podstawowych własności tokenów z danego tekstu

In [ ]:
import pandas as pd

def table(doc):
  table = []
  for tok in doc:
    tok_dic = {"form": tok.orth_, "lemma": tok.lemma_, "tag": ":".join([tok.tag_, tok._.feats]), "dep_label": tok.dep_, "dep_head": tok.head.orth_}
    table.append(tok_dic)
  return pd.DataFrame(table)

txt = "Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu.\
 Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych \
 wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii \
 (33 665 dolarów). Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka\
  będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki \
  polskiej. Nożyce między oboma krajami będą się więc rozwierać."

doc = nlp(txt)

tab = table(doc)

print(tab.to_string()) # prosty hack na wypisanie całej tabeli

###Czasami interesują nas większe całości niż pojedyncze tokeny, np. rzeczowniki często łączą się z przymiotnikami w frazy, żeby znajdować takie całości w tekście możemy korzystać z Matchera.

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
doc.is_tagged = True # tymczasowy hack, będzie zbędny w kolejnej wersji modelu
pattern = [{"TAG": "adj"}, {"TAG": "subst"}]
matcher.add("AdjSubst", None, pattern) # nazwa, funkcja, wzór

for match_id, start, end in matcher(doc):
  print(doc[start:end])

#### Niemniej w języku polskim, ze względu na swobodę szyku, takie rozwiązanie jest mało owocne, przymiotnik nie musi zawsze poprzedzać rzeczownika który określa, tego typu zależności są widoczne dopiero na poziomie analizy gramatycznej

### W gramatykach zależnościowych możliwa jest częściowa rekonstrukcja fraz rzeczownikowych (Noun Phrase - NP)
Aby to zrobić, musimy wydobyć z tekstu wszystkie rzeczowniki, zebrać wszystkie rozpinane przez nie poddrzewa, i wyrzucić ze zbioru te poddrzewa, które są już podfrazą większej frazy rzeczownikowej. To frazy rzeczownikowe, a nie same rzeczowniki, są jednostkami które mają dobrze określone znaczenie. Np. w zdaniu:

###*Żona Pawła jest blondynką.* 

nie mówimy o Pawle, ani o jakiejś abstrakcyjnej żonie, tylko o konkretnej kobiecie, którą wyodrębniamy przez fakt że jest żoną Pawła.

##Zadanie 3:

Napisz funkcję która na wejściu pobiera dokument, a na wyjściu zwraca listę fraz rzeczownikowych zgodnie z podanym powyżej opisem. Następnie wyodrębnij wszystkie frazy rzeczownikowe z tekstu pod zmienną txt.

In [ ]:
def get_nps(doc):
  # Wyszukuje w sparsowanym dokumencie wszystkie maksymalne frazy rzeczownikowe
  def is_contained(stree, stree_list):
    # Sprawdza czy podana fraza jest podfrazą innej, większej frazy
    return any([all([t in st for t in stree]) for st in stree_list])
  
  nouns = [t for t in doc if t.tag_ == "subst"] # to może być zależne od wersji niemorfeuszowej
  subtrees = [list(n.subtree) for n in nouns]
  to_eliminate = True
  while to_eliminate:
    # pętla do eliminowania fraz będących częścią większych fraz rzeczownikowych
    to_remove = []
    for st in subtrees:
      if is_contained(st, [x for x in subtrees if x != st]):
        to_remove.append(st)
    if to_remove == []:
      to_eliminate = False
    else:
      subtrees = [st for st in subtrees if st not in to_remove]
  return subtrees

for st in get_nps(nlp(txt)):
  print(st)

###Parser zależnościowy pozwala także na dzielenie dokumentów na zdania w sposób bardziej inteligentny, niż posługując się regułami interpunkcji.
###Za zdanie uważamy nieprzerwaną sekwencję tokenów które są powiązane relacjami zależnościowymi.
###Zdania są zapisane w atrybucie doc.sents dokumentu.

In [ ]:
for s in doc.sents:
  print(s)

displacy.render(doc, jupyter = True)

#Część czwarta - Rozpoznawanie jednostek nazewniczych (NER)
####Nasz model do spaCy wykorzystuje 6 rodzajów etykiet:
- placeName - miejsca antropogeniczne, np. Dania, Londyn
- geogName - naturalne miejsca geograficzne, np. Tatry, Kreta
- persName - imiona i nazwiska osób, np. J. F. Kennedy, gen. Maczek
- orgName - nazwy organizacji, np. NATO, Unia Europejska
- date - daty, np. 22 marca 1999, druga połowa kwietnia
- time - czas, np. 18:55, pięć po dwunastej

####Nie pozwala na wykrywanie zagnieżdżonych jednostek nazewniczych, np. [placeName: **aleja** [persName: **Piłsudskiego**]]

####Wykryte wzmianki są przechowywane w atrybucie doc.ents dokumentu, każda z tych wzmianek ma atrybut ent.label_, w którym przechowywana jest jej etykieta.

In [ ]:
print(doc, "\n\n")
for e in doc.ents:
  print(e, e.label_)


###displaCy pozwala także na wizualizację jednostek nazewniczych

In [ ]:
displacy.render(doc, style="ent", jupyter = True)

W obecnym modelu NER, uwzględnione są "uniwersalne" kategorie jednostek nazewniczych, jednak w zależności od zastosowania będziemy najprawdopodobniej potrzebowali innych kategorii - np. osobnej kategorii dla nazw aktów prawodawczych, lub kwot i walut.

Bez problemu można zastąpić domyślny model własnym, wytrenowanym przez CLI spaCy na własnych oznakowanych danych. Więcej informacji tu: https://spacy.io/api/cli#train

##Zadanie 4:

W zmiennej txt znajduje się dokument historyczny, wydobądź z niego wszystkie *zdania* zawierające daty, i po kolei je wypisz, rozważ możliwe sposoby automatycznego szeregowania dat wyrażonych w różny sposób (po ewentualnym sprowadzeniu ich do kanonicznej postaci). Rozwiązanie tego problemu jest trudne, i istnieją do niego odrębne narzędzia, umożliwiają one np. automatyczną rekonstrukcję chronologii wydarzeń.

In [ ]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/webinar_resources/master/3.txt").text

doc = nlp(txt)
events = []
for s in doc.sents:
  ents = s.ents
  labs = [e.label_ for e in ents]
  if "date" in labs:
    events.append(s)

for s in events:
  print(s)

Całym dokumentom także można przypisywać reprezentację wektorową, najprostszy sposób, to po prostu uśrednianie wektorów wszystkich tokenów w dokumencie.

Wektor reprezentujący dokument jest przechowywane w atrybucie doc.vector

In [ ]:
document_vector = doc.vector
mean_document_vector = sum([tok.vector for tok in doc])/len(doc)
print(document_vector == mean_document_vector)

In [1]:
import spacy
nlp = spacy.load("pl_spacy_model_morfeusz")

In [10]:
from sklearn.cluster import KMeans
import numpy

txt = """Lwy jako jedyne kotowate żyją w grupach rodzinnych. Stado składa się z wielu osobników – spokrewnionych samic, ich potomstwa obojga płci i kilku niespokrewnionych samców, których głównym zadaniem jest obrona terytorium i zapładnianie samic. Odnotowano tylko dwa przypadki obecności niespokrewnionych samic w stadzie W stadzie może być jeden samiec (typowe dla populacji z Tsavo) lub koalicja 2–4 samców. Większość stad liczy od 2 do 12 dorosłych osobników. Wielkość, liczba i zagęszczenie stad są uzależnione od dostępności pokarmu oraz liczebności lwów na danym obszarze.

Każde stado ma swoją własną hierarchię, w której najsłabszy samiec ma rangę wyższą od samic. Samce stale rywalizują o przywództwo nad stadem z innymi lwami i rzadko zdarza się, by samiec lwa żył w jednym stadzie dłużej niż 3 lata. Jeśli przewodzący samiec zostanie pokonany przez innego, odchodzi od stada i z reguły już nigdy do niego nie wraca.


Młode samice zwykle pozostają w stadzie na stałe, młode samce opuszczają stado po osiągnięciu dojrzałości płciowej.

Wyjaśnieniem stadnego – nietypowego dla kotowatych – trybu życia mogą być dwie przyczyny: zwiększenie skuteczności pozyskiwania pokarmu oraz zwiększenie przeżywalności młodych. Większa skuteczność polowania w stadzie została wielokrotnie potwierdzona obserwacjami. Badacze podnoszą jednak argument, że w większym stadzie zdobycz dzielona jest pomiędzy wiele osobników. Mniejsza skuteczność lwa polującego samotnie nagradzana jest natomiast większą porcją posiłku, jaka mu pozostaje bez konieczności dzielenia się ze współbiesiadnikami. Wprawdzie większość lwów żyje w stadach, ale samotniczy tryb życia nie jest u nich rzadkością. Poza tym lwy (zarówno samce, jak i samice) oddalają się czasem od stada, aby zapolować samotnie, co sugeruje, że wspólne polowania nie są najmocniejszym czynnikiem wzmacniającym relacje socjalne tych zwierząt.

W trakcie wieloletnich obserwacji lwów w Serengeti badacze stwierdzili, że największą przeżywalność młodych uzyskują stada, w których są co najmniej trzy współpracujące ze sobą lwice. Jedna lub dwie samice nie są w stanie uchronić młodych przed nowym samcem, którego pojawienie się w stadzie zagraża lwiątkom – lub przed drapieżnikami, kiedy samice oddaliły się w poszukiwaniu zdobyczy. Lwice wspólnie opiekują się wszystkimi młodymi ze stada, natomiast samiec chroniący terytorium i samic chroni jedynie swoje młode.

W obronie stada przed intruzami lwice stają razem z samcami. Najsilniejsze samce występują do przodu, a reszta stada staje za nimi.

Większość czasu spędzają odpoczywając – do 20 godzin w ciągu doby.

Lwy są mięsożercami polującymi głównie na ssaki kopytne, jak antylopy, zebry, gazele, bawoły afrykańskie, żyrafy i guźce zwyczajne oraz sporadycznie młode większych ssaków, takich jak słonie, hipopotamy czy nosorożce. Gdy brakuje pokarmu, zdarza im się napadać na zwierzęta hodowlane, zjadać gryzonie, ptaki czy padlinę, a nawet ludzi.

Na terenach otwartych lwy najczęściej polują w nocy. Tam, gdzie są wysokie trawy lub gęste krzewy, polowania zdarzają się również w ciągu dnia. Samce rzadko uczestniczą w polowaniach. Przyłączają się do łowów na dużego zwierza, takiego jak bawoły. Wówczas siła samca jest niezbędna do przytrzymania, a następnie powalenia ofiary ataku. Samice zwykle polują stadnie. Badania przeprowadzone przez G. Schallera w 1972 roku w Parku Narodowym Serengeti wykazały, że lwy mają dwukrotnie więcej szans na złapanie ofiary, gdy polują w grupie, niż samotnie (17% szans ma samica polująca pojedynczo, a 30% jeśli polują dwie). Kilka samic chętnie współpracuje ze sobą na polowaniu. Podczas polowania rozpraszają się na rozległym obszarze. Część z nich czeka, aż pozostałe nagonią zdobycz w ich stronę, wtedy zaczyna się atak.


Skuteczność ataków zależy od zdobyczy, pory dnia, rodzaju terenu oraz umiejętności współpracy w stadzie. W przypadku gazel skuteczność polowania wynosi ok. 14%, gnu i zebr 38%, a guźców 48%. Polowanie nocne charakteryzuje się 33% skutecznością, dzienne 21%. Na otwartych przestrzeniach (sawanna), gdzie ofiara ma sprzyjające warunki do ucieczki, skuteczność polowania zależy głównie od umiejętności współpracy pomiędzy osobnikami stada. Przeciętnie tylko 12 procent takich polowań kończy się sukcesem, a w buszu 41 procent. W zależności od terenu, upodobań oraz sposobu obrony stosowanego przez ofiarę lwy używają różnych technik łowieckich.


Podstawowa technika polega na spłoszeniu ofiary i zagonieniu jej do pułapki. Młode lwy kierują upatrzoną zdobycz ku ukrytym w krzakach lub w trawie bardziej doświadczonym osobnikom, które powalają zwierzę. Lwy zabijają, przewracając ofiarę na ziemię i dusząc ją, trzymając zwierzę za pysk lub zaciskając zęby na gardle. Jeśli samica jest zmuszona polować samotnie, podkrada się do ofiary na ile to jest możliwe, zwykle na odległość 30 m lub bliżej. Większość ssaków kopytnych potrafi biegać szybciej niż lwy.

Lwy, wykorzystując przewagę liczebną, często odbierają zdobycz innym drapieżnikom – gepardom, hienom, lampartom i likaonom. Upolowaną lub zdobytą zwierzyną posila się całe stado, a ponieważ każdy lew stara się zjeść jak najwięcej, dochodzi między nimi do utarczek. Najsilniejszy lew zjada najwięcej, po nim pozostałe samce, później samice, a na końcu młode. Dzienne zapotrzebowanie pokarmowe dorosłego samca sięga 7 kg, a samicy 5 kg mięsa.


Lwy rozmnażają się przez cały rok. Gdy samica wchodzi w ruję, jej gruczoły zaczynają wydzielać intensywną woń. Samiec wyczuwa ją węchem, w czym pomaga mu narząd Jacobsona. Samica wybiera partnera spośród dorosłych lwów ze stada, a następnie oddala się z nim na kilka dni. Przez ten czas para kopuluje kilkadziesiąt razy dziennie, w przerwach liżąc się i łasząc do siebie.

Podczas samej kopulacji samica leży, a samiec kuca nad nią, przytrzymując partnerkę zębami za kark.


Charakterystyczna „mina” jaką przybiera samiec, aby lepiej wyczuć samicę narządem Jacobsona
Młode rodzą się w liczbie od 1 do 6 (w większości przypadków od 1 do 4) po ciąży trwającej 100–114 dni. Nowo narodzone lwiątka mają cętki. Ważą od 1 do 2 kg. Młodymi opiekuje się całe stado. Często, gdy lwice ruszają na polowanie, lwiątka pozostają pod opieką jednej lub dwóch „ciotek”, które troszczą się o nie do powrotu matek. Jest to zachowanie niepowtarzalne w rodzinie kotowatych.

Tylko jedno na pięć lwiątek osiąga dojrzałość. Młodym zagrażają drapieżniki, takie jak sępy, hieny, czy lamparty, choroby, a także samce własnego gatunku, gdyż po objęciu przywództwa nad stadem lew zabija młode swego poprzednika, by samice szybciej mogły wejść w ruję i urodzić jego potomstwo. Najnowsze badania wykazują, że lwice mogą się temu zjawisku przeciwstawić. Samice pozwalają się obwąchiwać nowemu przywódcy stada. Wydzielają przy tym odpowiednie hormony zapachowe. Równocześnie prowadzą go do swojego legowiska, gdzie stężenie zapachu jest bardzo silne. Im dłużej lew wącha ten zapach, tym bardziej zmniejsza się jego agresja. Ten proces trwa od trzech do pięciu dni. Potem dzieci, dobrze ukryte, są już bezpieczne. Najmniejszą śmiertelność wśród młodych uzyskują samice rodzące w zbliżonym czasie, wspólnie opiekujące się młodymi i wspólnie je karmiące.

Małe lwy rodzą się ślepe, ich oczy otwierają się około 3–11 dnia. W wieku 10–15 dni łapy młodych są już dość silne i zwierzęta zaczynają chodzić, choć na pełną sprawność muszą poczekać jeszcze 3-4 tygodnie. Do ósmego tygodnia życia matka trzyma je w ukryciu. Miesięcznym lwom wyrastają pierwsze zęby.

W wieku 1-2 miesięcy lwiątka zaczynają oddalać się od matki i bawić się z innymi, dziecięce plamy na futrze znikają w wieku 3 miesięcy. Lwy zaczynają polować w wieku 11 miesięcy, po kolejnych 5 miesiącach są już w pełni samodzielne. Lwia grzywa zaczyna rosnąć w wieku 2 lat. Samce są dojrzałe płciowo po 2,5 roku, samice w wieku 2,5–4 lat.

Samce żyją krócej niż samice. Jeśli wcześniej nie zginą w walce, mogą dożyć wieku ponad 10 lat. Notowano pojedyncze przypadki 16-letnich samców żyjących na wolności. Taki wiek, 15–16 lat, dla samic jest natomiast wiekiem przeciętnym. W Parku Serengeti samice żyją do 18 lat. Przeciętny wiek życia w niewoli wynosi 13 lat. Najstarszy znany lew żył 30 lat.
"""


In [25]:
sents = list(nlp(txt).sents)
print(len(sents))
print(nlp("84")[0].tag_)

cleared_sents = []
for s in sents:
    tokens = []
    for t in s:
        if t.tag_ not in ["num"] and not t.is_stop:
            tokens.append(t.orth_)
    cleared_sents.append(tokens)

word_scores = {}

for s in cleared_sents:
    for word in s:
        if word in word_scores:
            word_scores[word] += 1
        else:
            word_scores[word] = 1

sent_scores = []
for cs, s in zip(cleared_sents, sents):
    score = sum([word_scores[w] for w in cs])#/len(cs)
    sent_scores.append((score,s))

for s in sorted(sent_scores):
    print(s)

85
num
(90, Tylko jedno na pięć lwiątek osiąga dojrzałość.)
(91, Ważą od 1 do 2 kg.)
(91, Wydzielają przy tym odpowiednie hormony zapachowe.)
(92, Jest to zachowanie niepowtarzalne w rodzinie kotowatych.)
(93, Nowo narodzone lwiątka mają cętki.)
(93, Ten proces trwa od trzech do pięciu dni.)
(93, Miesięcznym lwom wyrastają pierwsze zęby.)
(95, Samce rzadko uczestniczą w polowaniach.)
(95, Lwy rozmnażają się przez cały rok.)
(95, Młodymi opiekuje się całe stado.)
(96, Do ósmego tygodnia życia matka trzyma je w ukryciu.)
(97, Samice zwykle polują stadnie.)
(97, Kilka samic chętnie współpracuje ze sobą na polowaniu.)
(97, Podczas polowania rozpraszają się na rozległym obszarze.)
(97, Podstawowa technika polega na spłoszeniu ofiary i zagonieniu jej do pułapki.)
(97, Notowano pojedyncze przypadki 16-letnich samców żyjących na wolności.)
(98, Większość stad liczy od 2 do 12 dorosłych osobników.)
(100, Lwy jako jedyne kotowate żyją w grupach rodzinnych.)
(102, Samice pozwalają się obwąchiwać 

In [38]:
from sklearn.metrics import pairwise_distances
from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power

sents = list(nlp(txt).sents)
vex = numpy.array([s.vector for s in sents])
weights = [1 for s in sents]
similarities = 1- pairwise_distances(vex, vex, metric="cosine")
n = len(sents)
print(n)
G = sparse.csr_matrix((weights, (similarities[:,0], similarities[:,1])), shape=(n,n))
#G = sparse.csr_matrix()
print(dists)
pr = pagerank(G)
rank = []
for s, p in zip(sents, pr):
    rank.append((p,s))

srt = sorted(rank, reverse=True)

85


ValueError: negative row index found

In [16]:
med_sents = [s for s in sents if len(s) > 8 and len(s) < 25]
vex = numpy.array([s.vector for s in med_sents])
print(vex.shape)

clusterer = KMeans(n_clusters = 8)
labels = clusterer.fit_predict(vex)
sents_by_label = {l: [s for i, s in enumerate(med_sents) if labels[i] == l] for l in set(labels)}

(55, 100)


In [ ]:
sents = 

In [17]:
for l in set(labels):
    print("\n")
    for s in sents_by_label[l]:
        print(s)
    print("\n")



Gdy samica wchodzi w ruję, jej gruczoły zaczynają wydzielać intensywną woń.
Samiec wyczuwa ją węchem, w czym pomaga mu narząd Jacobsona.
Im dłużej lew wącha ten zapach, tym bardziej zmniejsza się jego agresja.




Wielkość, liczba i zagęszczenie stad są uzależnione od dostępności pokarmu oraz liczebności lwów na danym obszarze.
Każde stado ma swoją własną hierarchię, w której najsłabszy samiec ma rangę wyższą od samic.
Wyjaśnieniem stadnego – nietypowego dla kotowatych – trybu życia mogą być dwie przyczyny: zwiększenie skuteczności pozyskiwania pokarmu oraz zwiększenie przeżywalności młodych.
Większa skuteczność polowania w stadzie została wielokrotnie potwierdzona obserwacjami.
Badacze podnoszą jednak argument, że w większym stadzie zdobycz dzielona jest pomiędzy wiele osobników.
Mniejsza skuteczność lwa polującego samotnie nagradzana jest natomiast większą porcją posiłku, jaka mu pozostaje bez konieczności dzielenia się ze współbiesiadnikami.
Wówczas siła samca jest niezbędna do pr